<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Todo-list" data-toc-modified-id="Todo-list-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Todo list</a></span></li><li><span><a href="#1.-부재종-후보군-Dataframe-생성" data-toc-modified-id="1.-부재종-후보군-Dataframe-생성-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. 부재종 후보군 Dataframe 생성</a></span></li></ul></div>

In [1]:
import pandas as pd
from tqdm import tqdm
from haversine import haversine

# 1. 부재종 후보군 Dataframe 생성
1. H.convergens
2. C7
3. H.Parenthesis


주변에 c9이 있는지 없는지부터검사.


In [116]:
df = pd.read_csv('./Data/20220209_07_21_무당벌레관측.csv')
df_c9 = pd.read_csv('./20220209_bipunctata_arround_gps_1000.csv')

In [117]:
len(df.index)
df = df[(df.State!='Nuevo León')&(df.State!='Sonora')&(df.State!='Chihuahua')&(df.State!='Puebla')&
      (df.State!='Francisco Morazán')&(df.State!='Coahuila')&(df.State!='Chihuahua')&(df.State!='Durango')&
       (df.State!='San Luis Potosí')&(df.State!='Nayarit')&(df.State!='Veracruz')&(df.State!='México')&(df.State!='Jalisco')
      &(df.State!='Baja California')&(df.State!='District of Columbia')&(df.State!='Tamaulipas')
       &(df.State!='Michoacán')&(df.State!='Tlaxcala')&(df.State!='Oaxaca')&(df.State!='Morelos')
      &(df.State!='Guatemala')&(df.State!='Distrito Federal')&(df.State!='Hidalgo')        
      &(df.State!='Guanajuato')&(df.State!='Querétaro')].reset_index(drop=True)

len(df.index)


len(df.index)
df = df[(df.State!='Sinaloa')&(df.State!='Campeche')&(df.State!='Chihuahua')&(df.State!='Puebla')&
      (df.State!='Nicaragua')&(df.State!='Quintana Roo')&(df.State!='Morovis')&(df.State!='Cabo Rojo')&
       (df.State!='Chiapas')&(df.State!='La Libertad')&(df.State!='Laborie')&(df.State!='México')&(df.State!='Jalisco')
      &(df.State!='Guayanilla')&(df.State!='Saint-Pierre')&(df.State!='Tamaulipas')
       &(df.State!='Cayey')&(df.State!='Cortés')&(df.State!='Oaxaca')&(df.State!= 'León')
      &(df.State!='Guatemala')&(df.State!='Guerrero')&(df.State!='Hidalgo')        
      &(df.State!='Caguas')&(df.State!='Zacatecas')].reset_index(drop=True)

len(df.index)


len(df.index)
df = df[(df.State!='Sinaloa')&(df.State!='Campeche')&(df.State!='Chihuahua')&(df.State!='Puebla')&
      (df.State!='Yucatán')&(df.State!='Petén')&(df.State!='Morovis')&(df.State!='Tabasco')&
       (df.State!='El Paraíso')&(df.State!='Lempira')&(df.State!='Laborie')&(df.State!='México')&(df.State!='Jalisco')
      &(df.State!='Guayanilla')&(df.State!= 'Masaya')&(df.State!='Baja California Sur')
       &(df.State!='Choluteca')&(df.State!='Saba')&(df.State!='Oaxaca')&(df.State!= 'León')
      &(df.State!='Guatemala')&(df.State!='Patillas')&(df.State!='Corozal')        
      &(df.State!='Aguascalientes')&(df.State!='Saint-Martin et Saint-Barthélémy')].reset_index(drop=True)

len(df.index)

200975

In [118]:
#gps 변환
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
df['gps'] = df_gps



#c9 gps 변환
df_c9.Latitude = df_c9.Latitude.astype(float)
df_c9.Longitude = df_c9.Longitude.astype(float)

df_c9_gps = []
for i in tqdm(range(len(df_c9))):
    df_c9_gps.append( (df_c9.Latitude[i], df_c9.Longitude[i]) )
    
df_c9['gps'] = df_c9_gps

100%|███████████████████████████████████████████████████████████████████████████| 1487/1487 [00:00<00:00, 67789.03it/s]


In [119]:
#gps 정확도가 1km이상 오차가 있으면 거르기
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)

In [120]:
#Latitude, Longitude 0으로 기록된 부분들 삭제
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)

In [121]:
# Absence 종으로 선정될 수 있는 종들만 고르기
df_ab = df[(df.Species=='Harmonia axyridis')|(df.Species=='Coccinella septempunctata')].reset_index(drop=True)

In [122]:
df_ab['c9_500m'] = 0

In [123]:
df_ab = df_ab[df_ab.Longitude<=-87.5].drop_duplicates().reset_index(drop=True)

In [124]:
for i in tqdm(range(len(df_ab))): # c9만 뽑아놓은 데이터 프레임
    for j in range(len(df_c9.gps)):  # 전체 무당벌레 데이터 프레임        
        if haversine(df_ab.gps[i], df_c9.gps[j], unit='km') <= 0.5 and df_c9.Year[j] == df_ab.Year[i]:
            df_ab['c9_500m'][i] += 1



  0%|                                                                                        | 0/48551 [00:00<?, ?it/s]<ipython-input-124-1a53204b015c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ab['c9_500m'][i] += 1
100%|████████████████████████████████████████████████████████████████████████████| 48551/48551 [21:53<00:00, 36.97it/s]


In [146]:
# 주가 없다면 거르기
df_ab = df_ab[df_ab.State.isnull()==False].reset_index(drop=True)

In [147]:
hip = df_ab[df_ab.Species=='Harmonia axyridis'].sort_values(by=['Year','Latitude','Longitude']).reset_index(drop=True)
sep = df_ab[df_ab.Species=='Coccinella septempunctata'].sort_values(by=['Year','Latitude','Longitude']).reset_index(drop=True)


In [148]:
hip.State = hip.State.astype(str)
sep.State = sep.State.astype(str)

In [149]:
one_idx=[]
many_idx = []
for i in hip.State.unique():
        if i == 'Texas':
#             print(i,"삽입전 길이",len(many_idx))
#             print(i,"의 개수",len(hip[hip.State==i].index))
#             for j in range(len(hip[hip.State==i].index)):
#                 if (j%120) == 0:
#                     many_idx.append(hip[hip.State==i].index[j])
#             print(i,"삽입된 후 길이",len(many_idx))
            pass

        elif i =='California':
            pass
        elif len(hip[(hip.State == i)]) <=20:
            one_idx.append(hip[(hip.State == i)].index)
                        
        elif len(hip[(hip.State == i)])<=60:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%4) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(hip[(hip.State == i)])<=120:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%6) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(hip[(hip.State == i)])<=500:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%33) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(hip[(hip.State == i)])<=1000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%63) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(hip[(hip.State == i)])<=2000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%73) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(hip[(hip.State == i)])<=4000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%63) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
                        
        else:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(hip[hip.State==i].index))
            for j in range(len(hip[hip.State==i].index)):
                if (j%83) == 0:
                    many_idx.append(hip[hip.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))

ff = [element for array in one_idx for element in array]      


hip_1 = hip.iloc[many_idx]
hip_2 = hip.iloc[ff]

hip_fin = pd.concat([hip_1,hip_2]).reset_index(drop=True)

Louisiana 삽입전 길이 0
Louisiana 의 개수 709
Louisiana 삽입된 후 길이 12
Alabama 삽입전 길이 12
Alabama 의 개수 218
Alabama 삽입된 후 길이 19
Tennessee 삽입전 길이 19
Tennessee 의 개수 202
Tennessee 삽입된 후 길이 26
Oklahoma 삽입전 길이 26
Oklahoma 의 개수 496
Oklahoma 삽입된 후 길이 42
Missouri 삽입전 길이 42
Missouri 의 개수 855
Missouri 삽입된 후 길이 56
Illinois 삽입전 길이 56
Illinois 의 개수 1434
Illinois 삽입된 후 길이 76
Wisconsin 삽입전 길이 76
Wisconsin 의 개수 749
Wisconsin 삽입된 후 길이 88
Minnesota 삽입전 길이 88
Minnesota 의 개수 1187
Minnesota 삽입된 후 길이 105
Washington 삽입전 길이 105
Washington 의 개수 917
Washington 삽입된 후 길이 120
British Columbia 삽입전 길이 120
British Columbia 의 개수 95
British Columbia 삽입된 후 길이 136
Mississippi 삽입전 길이 136
Mississippi 의 개수 399
Mississippi 삽입된 후 길이 149
Arizona 삽입전 길이 149
Arizona 의 개수 21
Arizona 삽입된 후 길이 155
Arkansas 삽입전 길이 155
Arkansas 의 개수 405
Arkansas 삽입된 후 길이 168
Kansas 삽입전 길이 168
Kansas 의 개수 172
Kansas 삽입된 후 길이 174
Nevada 삽입전 길이 174
Nevada 의 개수 24
Nevada 삽입된 후 길이 180
Colorado 삽입전 길이 180
Colorado 의 개수 348
Colorado 삽입된 후 길이 191
Utah 삽입전 길이 191
Utah 의 개

In [150]:
one_idx=[]
many_idx = []

In [133]:
for i in sep.State.unique():
        if i == 'California' or i == 'Texas':
#             print(i,"삽입전 길이",len(many_idx))
#             print(i,"의 개수",len(sep[sep.State==i].index))
#             for j in range(len(sep[sep.State==i].index)):
#                 if (j%100) == 0:
#                     many_idx.append(sep[sep.State==i].index[j])
#             print(i,"삽입된 후 길이",len(many_idx))
            pass
            
        elif len(sep[(sep.State == i)]) <=10:
            one_idx.append(sep[(sep.State == i)].index)
            
            
        elif len(sep[(sep.State == i)])<=60:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%4) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(sep[(sep.State == i)])<=120:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%6) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(sep[(sep.State == i)])<=200:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%11) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(sep[(sep.State == i)])<=500:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%33) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
            
        elif len(sep[(sep.State == i)])<=1000:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%63) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
            
        elif len(sep[(sep.State == i)])<=2500:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%73) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))
        else:
            print(i,"삽입전 길이",len(many_idx))
            print(i,"의 개수",len(sep[sep.State==i].index))
            for j in range(len(sep[sep.State==i].index)):
                if (j%63) == 0:
                    many_idx.append(sep[sep.State==i].index[j])
            print(i,"삽입된 후 길이",len(many_idx))

        

Louisiana 삽입전 길이 0
Louisiana 의 개수 68
Louisiana 삽입된 후 길이 12
Arizona 삽입전 길이 12
Arizona 의 개수 168
Arizona 삽입된 후 길이 28
Alabama 삽입전 길이 28
Alabama 의 개수 33
Alabama 삽입된 후 길이 37
New Mexico 삽입전 길이 37
New Mexico 의 개수 115
New Mexico 삽입된 후 길이 57
Colorado 삽입전 길이 57
Colorado 의 개수 833
Colorado 삽입된 후 길이 71
Illinois 삽입전 길이 71
Illinois 의 개수 198
Illinois 삽입된 후 길이 89
Wisconsin 삽입전 길이 89
Wisconsin 의 개수 126
Wisconsin 삽입된 후 길이 101
Washington 삽입전 길이 101
Washington 의 개수 610
Washington 삽입된 후 길이 111
British Columbia 삽입전 길이 111
British Columbia 의 개수 58
British Columbia 삽입된 후 길이 126
Alberta 삽입전 길이 126
Alberta 의 개수 127
Alberta 삽입된 후 길이 138
Manitoba 삽입전 길이 138
Manitoba 의 개수 47
Manitoba 삽입된 후 길이 150
Mississippi 삽입전 길이 150
Mississippi 의 개수 53
Mississippi 삽입된 후 길이 164
Oklahoma 삽입전 길이 164
Oklahoma 의 개수 313
Oklahoma 삽입된 후 길이 174
Kansas 삽입전 길이 174
Kansas 의 개수 161
Kansas 삽입된 후 길이 189
Missouri 삽입전 길이 189
Missouri 의 개수 157
Missouri 삽입된 후 길이 204
Utah 삽입전 길이 204
Utah 의 개수 415
Utah 삽입된 후 길이 217
Iowa 삽입전 길이 217
Iowa 의 개수 30
Iowa 삽

In [134]:
ff = [element for array in one_idx for element in array]

In [135]:
sep_1 = sep.iloc[many_idx]
sep_2 = sep.iloc[ff]
sep_fin = pd.concat([sep_1,sep_2]).reset_index(drop=True)

In [136]:
df_fin = pd.concat([sep_fin,hip_fin]).reset_index(drop=True)

In [137]:
df_fin.to_csv("absence_지역선정_new.csv")

In [138]:
df_origin = pd.read_csv("./DOA_ML/ML_DATA/220220_DOA_bi(최종,minmax).csv")

In [139]:
df_origin = df_origin[(df_origin.Species!='Adalia bipunctata')&(df_origin.Species!='Hippodamia convergens')].reset_index(drop=True)

In [140]:
idx_drop = []

for i in df_fin.gps:
    for j in df_origin.gps:
        if i == j:
            idx_drop.append(i)

In [141]:
df_fin = df_fin.iloc[:,2:-1]

In [142]:
df_origin = df_origin.iloc[:,1:9]

In [143]:
df_end = pd.concat([df_fin,df_origin]).reset_index(drop=True)

In [144]:
df_end.sort_values(by='Year').reset_index(drop=True)

,Source,Year,Species,Latitude,Longitude,public_positional_accuracy,State,gps
0,Bugguide.net,2007,Coccinella septempunctata,29.935238,-90.151400,0.0,Louisiana,"(29.935238, -90.1514)"
1,Bugguide.net,2007,Harmonia axyridis,29.935238,-90.151400,0.0,Louisiana,"(29.935238, -90.1514)"
2,Bugguide.net,2007,Harmonia axyridis,29.935238,-90.151400,0.0,Louisiana,"(29.935238, -90.1514)"
3,The Lost Ladybugs,2007,Harmonia axyridis,30.694357,-88.043054,0.0,Alabama,"(30.6943566, -88.043054)"
4,I-naturalist,2007,Harmonia axyridis,31.168473,-83.360076,4.0,Georgia,"(31.1684725053, -83.3600760064)"
...,...,...,...,...,...,...,...,...
3027,I-naturalist,2021,Coccinella septempunctata,39.983276,-74.132660,25.0,New Jersey,"(39.9832763672, -74.1326599121)"
3028,I-naturalist,2021,Coccinella septempunctata,39.687940,-74.834600,15.0,New Jersey,"(39.68794, -74.8346)"
3029,I-naturalist,2021,Coccinella septempunctata,38.253962,-85.716988,22.0,Kentucky,"(38.2539619796, -85.7169881747)"
3030,I-naturalist,2021,Coccinella septempunctata,45.101389,-93.038611,183.0,Minnesota,"(45.1013889, -93.0386111)"
